In [5]:
from data_loader import load_small_data
from gurobipy import *


In [108]:
instance = load_small_data()
print(instance)

{'horizon': 5, 'qualifications': ['A', 'B', 'C'], 'staff': [{'name': 'Olivia', 'qualifications': ['A', 'B', 'C'], 'vacations': []}, {'name': 'Liam', 'qualifications': ['A', 'B'], 'vacations': [1]}, {'name': 'Emma', 'qualifications': ['C'], 'vacations': [2]}], 'jobs': [{'name': 'Job1', 'gain': 20, 'due_date': 3, 'daily_penalty': 3, 'working_days_per_qualification': {'A': 1, 'B': 1, 'C': 1}}, {'name': 'Job2', 'gain': 15, 'due_date': 3, 'daily_penalty': 3, 'working_days_per_qualification': {'A': 1, 'B': 2}}, {'name': 'Job3', 'gain': 15, 'due_date': 4, 'daily_penalty': 3, 'working_days_per_qualification': {'A': 1, 'C': 2}}, {'name': 'Job4', 'gain': 20, 'due_date': 3, 'daily_penalty': 3, 'working_days_per_qualification': {'B': 2, 'C': 1}}, {'name': 'Job5', 'gain': 10, 'due_date': 5, 'daily_penalty': 3, 'working_days_per_qualification': {'C': 2}}]}


In [111]:
def create_vacations_staff(data):
    vacations = list()
    for employe in data['staff']:
        vacation_employe = list()
        for day in range (horizon):
            if (day+1) in employe['vacations']:
                vacation_employe.append(1)
            else: 
                vacation_employe.append(0)
        vacations.append(vacation_employe)
    return vacations

vacations = create_vacations_staff(instance)
print(vacations)

[[0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0]]


In [ ]:
for employe in range (nb_person):
    for competence un range (nb_competence):
        for time in range (T):
            m.addConstr(projects_through_time == planning[employe][time][project][competence], name="evolution_of_projects_constraints")
            
            
            

In [112]:
list_pers = [p['name'] for p in instance['staff']]
list_comp = instance['qualifications']
list_job = [p['name'] for p in instance['jobs']]

L = []
for pers in list_pers:
    for t in range(horizon):
        for p in list_job:
            for c in list_comp:
                L.append((pers, t, p, c))

possibility = tuplelist(L)

L2 = []

for t in range(horizon):
    for p in list_job:
        for c in list_comp:
            L2.append((t, p, c))

possibility_reduced = tuplelist(L2)

L3 = []

for t in range(horizon):
    for p in list_job:
        L3.append((t, p))

possibility_reduced2 = tuplelist(L3)

m = Model()

X = m.addVars(possibility, name='x', vtype=GRB.BINARY)

S = m.addVars(possibility_reduced, lb = -1000, name='s')

LP_MIN = m.addVars(possibility_reduced2, lb = -1000, name='lpm')

LP = m.addVars(possibility_reduced2, lb = -1000, name='lp')

m.update()



m.addConstrs(X.sum(pers,t,'*','*') <= 1 for pers in list_pers for t in range(horizon))


qualifs = create_qualifications(instance)
m.addConstrs(qualifs[i][j] >= X[pers,t,job,comp] for i,pers in enumerate(list_pers) for t in range(horizon) for j,comp in enumerate(list_comp) for job in list_job)


#m.addConstrs(S[T, job, comp] == (quicksum(X[pers, t, job, comp] for t in range(T) for pers in list_pers) - CP[j][q]) for j,job in enumerate(list_job) for q,comp in enumerate(list_comp))
#m.addConstrs(LP_MIN[T, job] == min_(S.select(T, job, '*')) for job in list_job for T in range(horizon))
#m.addConstrs((LP_MIN[T, job] == min_([(quicksum(X[pers, t, job, comp] for t in range(T) for pers in list_pers) - CP[j][q]) for q,comp in enumerate(list_comp)]))  for j,job in enumerate(list_job) for T in range(horizon))





#avancement du projet
m.addConstrs((LP_MIN[T, job] == (quicksum(X[pers, t, job, comp] for t in range(T) for pers in list_pers for comp in list_comp) - quicksum(CP[index_job][index_comp] for index_comp in range(nb_comp)) + 1))  for index_job, job in enumerate(list_job) for T in range(horizon))

#contrainte sur les compétences : pas plus que nécessaire
m.addConstrs(CP[index_job][index_comp] >= quicksum(X[pers,t,job,comp] for pers in list_pers for t in range(horizon)) for index_comp,comp in enumerate(list_comp) for index_job,job in enumerate(list_job))

#construction of LP
m.addConstrs(LP[T, job] == max_(LP_MIN[T, job],0) for T in range(horizon) for job in list_job)

m.update()

# gain function

# Fonction Objectif
m.setObjective(X.sum('*','*','*','*'), GRB.MAXIMIZE)  

# Résolution du PL
m.optimize()

NameError: name 'create_qualifications' is not defined

In [7]:
def create_staff_qualifications(data):
    E = [] #Staff qualifications
    for staff_member in data['staff']:
        E.append(staff_member['qualifications'])
    return E

create_staff_qualifications(toy_instance)

[['A', 'B', 'C'], ['A', 'B'], ['C']]

In [19]:
def working_days_per_qualification(data):
    CP = list() #Staff qualifications
    for job in data['jobs']:
        CPi = list()
        for qualification in data['qualifications']:
            if qualification in job['working_days_per_qualification']:
                CPi.append(job['working_days_per_qualification'][qualification])
            else:
                CPi.append(0)
        CP.append(CPi)
    return CP

qualification_per_job = working_days_per_qualification(toy_instance)
print(qualification_per_job)

[[1, 1, 1], [1, 2, 0], [1, 0, 2], [0, 2, 1], [0, 0, 2]]


In [39]:
def is_project_finished(theorie, pratic):
    finished_project = list()
    for i in range (len(theorie)): 
        finished_project_i = list()
        for j in range (len(theorie[i])):
            if theorie[i][j]<=pratic[i][j]:
                finished_project_i.append(1)
            else:
                finished_project_i.append(0)
        finished_project.append(min(finished_project_i))
    return finished_project

In [41]:
pratique = [[1, 1, 0], [1, 2, 0], [1, 0, 2], [0, 2, 1], [0, 0, 2]]
is_project_finished(qualification_per_job,pratique)

[0, 1, 1, 1, 1]

In [1]:
m = Model("Simple PL modelling")
# Création de 2 variables continues v0 et v1

planning=[]
nb_pers = len(instance['staff'])
nb_proj = len(instance['jobs'])
nb_comp = len(instance['qualication'])
horizon = instance['horizon']

for pers in instance['staff']:
    row = []
    for t in range(horizon)
        row.append({'competence':m.addVar(lb=0,ub=nb_comp-1,vtype=GRB.INTEGER, name=pers['name']+''+str(t)+'_competence'),'projet':m.addVar(lb=0,ub=nb_proj-1,vtype=GRB.INTEGER, name=pers['name']+''+str(t)+'_projet')})
    planning.append(row)

    


    
      
# maj du modèle
m.update()

# Ajout de 3 constraintes 
def get_finish_project(plan):
    jobs = instance['jobs']
    for t in range(horizon):
        for p in range(nb_pers):
            plan[]

for i in range(len(instance['staff'])):
    for t in range(instance['horizon'])
        row.append(m.addVar(lb=0,ub=1,vtype=GRB.INTEGER))
    planning.append(row)

      
# maj du modèle
m.update()

# Ajout de 3 constraintes 
def gain(L_proj,instance):
    s = 0
    for i in range(len(L_proj)):
        s = s + L_proj[i]*instance['jobs'][i]['gain']
    return s

def nb_proj(L_proj):
    return sum(L_proj)

m.addConstr(nb_proj(L) == 3)

m.update()
# Fonction Objectif
m.setObjective(gain(L,instance), GRB.MAXIMIZE)  

# Paramétrage (mode mute)
#m.params.outputflag = 0



#m.update()
# Affichage en mode texte du PL
print(m.display())


# Résolution du PL
m.optimize()

SyntaxError: invalid syntax (332830158.py, line 8)

In [53]:
m = Model("Simple PL modelling")
# Création de 2 variables continues v0 et v1

planning=[]
nb_pers = len(instance['staff'])
nb_proj = len(instance['jobs'])
nb_comp = len(instance['qualifications'])
horizon = instance['horizon']

 
def working_days_per_qualification(data):
    CP = list() 
    for job in data['jobs']:
        CPi = list()
        for qualification in data['qualifications']:
            if qualification in job['working_days_per_qualification']:
                CPi.append(job['working_days_per_qualification'][qualification])
            else:
                CPi.append(0)
        CP.append(CPi)
    return CP

working_days_per_qualification = working_days_per_qualification(instance)

for pers in instance['staff']:
    row = []
    for t in range(horizon):
        row.append({'competence':m.addVar(lb=0,ub=nb_comp-1,vtype=GRB.INTEGER, name=pers['name']+''+str(t)+'_competence'),'project':m.addVar(lb=0,ub=nb_proj-1,vtype=GRB.INTEGER, name=pers['name']+''+str(t)+'_project')})
    planning.append(row)



      
# maj du modèle
m.update()

# Ajout de 3 constraintes 

def is_project_finished(theorie, pratic):
    finished_project = list()
    for i in range (len(theorie)): 
        finished_project_i = list()
        for j in range (len(theorie[i])):
            if theorie[i][j]<=pratic[i][j]:
                finished_project_i.append(1)
            else:
                finished_project_i.append(0)
        finished_project.append(min(finished_project_i))
    return finished_project


def get_finish_project(plan, CP):
    res = [-1 for x in range(nb_proj)]
    jobs = instance['jobs']
    comp_done = [[0 for x in range(nb_comp)] for y in range(nb_proj)]
    for t in range(horizon):
        for p in range(nb_pers):
            pers = plan[p][t]
            print(type(pers['project']))
            comp_done[pers['project']][pers['competence'].x] += 1
            list_finish = is_project_finish(CP, comp_done)
            for i in range(len(list_finish)):
                if proj[i]==1 and res[i] != 1:
                    res[i]=t
    return res
        
def compute_gain(projects_done, data):
    gain = 0
    for i in range(len(projects_done)):
        g = projects_done[i]
        job = data['jobs'][i]
        if g > -1:
            gain += job['gain'] + min(0, (job['due_date'] - g) * job['daily_penalty'])
    return gain    
    
def gain(plan):
    timeline_project = get_finish_project(plan, CP)
    g = compute_gain(timeline_project, instance)
    return g

#m.addConstr(cgain() == 3)

m.update()
# Fonction Objectif
m.setObjective(gain(planning), GRB.MAXIMIZE)  

# Paramétrage (mode mute)
#m.params.outputflag = 0



#m.update()
# Affichage en mode texte du PL
print(m.display())


# Résolution du PL
m.optimize()

<class 'gurobipy.Var'>


TypeError: list indices must be integers or slices, not Var

In [ ]:
for i in range(nb_proj):
    for t in range(horizon):
        m.addConstr(progress(t,i,c) >= working_days_per_qualification[i][c]  >> L_P[i][t] == 1 for c in range(nb_comp))
        
def progress(T,p,c):
    progress=0
    for t in range(T):
        for i in range(nb_pers):
            progress += planning[i][t][p][c]
            
            
def transfer_gain():
    res = []
    for i in range(nb_proj):
        res.append(horizon - sum(L_P[i]) + 1)
    return res

In [107]:
m = Model("Simple PL modelling")
# Création de 2 variables continues v0 et v1

planning=[]
nb_pers = len(instance['staff'])
nb_proj = len(instance['jobs'])
nb_comp = len(instance['qualifications'])
horizon = instance['horizon']

 
def working_days_per_qualification(data):
    CP = list() 
    for job in data['jobs']:
        CPi = list()
        for qualification in data['qualifications']:
            if qualification in job['working_days_per_qualification']:
                CPi.append(job['working_days_per_qualification'][qualification])
            else:
                CPi.append(0)
        CP.append(CPi)
    return CP

CP = working_days_per_qualification(instance)


for pers in instance['staff']:
    row = []
    for t in range(horizon):
        proj=[]
        for p in range(nb_proj):
            comp = []
            for c in range(nb_comp):
                comp.append(m.addVar(lb=0,ub=1,vtype=GRB.INTEGER, name=pers['name']+'_time:'+str(t)+'_comp:'+str(c)+'_proj:'+str(p)))
            proj.append(comp)      
        row.append(proj)
    planning.append(row)
    


L_P = []
for i in range(nb_proj):
    row = []
    for t in range(horizon):                     
        row.append(m.addVar(lb=0,ub=1,vtype=GRB.INTEGER, name='Project:'+str(i)+'_time:'+str(t)))
    L_P.append(row)
      
# maj du modèle
m.update()

        
def progress(plan,T,p,c):
    prog=0
    for t in range(T):
        for i in range(nb_pers):
            prog += plan[i][t][p][c]
    return prog

#for i in range(nb_proj):
    #for t in range(horizon):
        #m.addConstr((progress(planning,t,i,0) >= CP[i][0])  >> (L_P[i][t] == 1))
        #m.addConstr((plan[i][t][0][0] >= CP[i][0])  >> (L_P[i][t] == 1))

#m.addConstr(planning[0][0][0][0]==1)
gain_min = m.addMVar(nb_proj)
for i in range(nb_proj):
    #print(L_P[i])

    #m.addConstr((gain_min[i] == min_(instance['jobs'][i]['due_date'] - horizon + 1, 0)))
    m.addConstr((gain_min[i] == min_(instance['jobs'][i]['due_date'] + horizon + 1, constant =  0)))


# objective function

def transfer_gain(LP):
    res = []
    for i in range(nb_proj):
        res.append(horizon - sum(LP[i]) + 1)
    return res

def compute_gain(projects_done, data):
    gain = 0
    for i in range(len(projects_done)):
        g = projects_done[i]
        job = data['jobs'][i]
        #if g <= horizon :
        # gain += job['gain'] + min(0, (job['due_date'] - g) * job['daily_penalty'])     
        #print(gain_min[i])
        #gain += min_(horizon + 1 - g, 1)*(job['gain'] + gain_min[i] * job['daily_penalty'])  
        gain += (horizon + 1 - g)*(job['gain'] + (job['due_date'] - g) * job['daily_penalty'])  
    return gain

def gain(LP):
    return compute_gain(transfer_gain(LP),instance)
    

m.update()
# Fonction Objectif
m.setObjective(gain(L_P), GRB.MAXIMIZE)  


# Résolution du PL
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 255 columns and 0 nonzeros
Model fingerprint: 0x93902210
Model has 75 quadratic objective terms
Model has 5 general constraints
Variable types: 5 continuous, 250 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [6e+00, 1e+01]
  QObjective range [6e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 595.0000000
Presolve removed 0 rows and 255 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 595 -0 

Optimal solution found

In [ ]:
# Create variables
x = m.addVar(ub=10, vtype=GRB.CONTINUOUS, name="x")
y = m.addVar(ub=5, vtype=GRB.CONTINUOUS, name="y")
gain = m.addVar(vtype=GRB.CONTINUOUS, name="z")
w1 = m.addVar(vtype=GRB.CONTINUOUS, name="w1")
w2 = m.addVar(vtype=GRB.CONTINUOUS, name="w2")
b = m.addVar(vtype=GRB.BINARY, name="b")

# Constants
eps = 0.0001
M = 10 + eps # smallest possible given bounds on x and y

# Model if x > y, then b = 1, otherwise b = 0
m.addConstr(x >= y + eps - M * (1 - b), name="bigM_constr1")
m.addConstr(x <= y + M * b, name="bigM_constr2")

# Add indicator constraints
m.addConstr((b == 1) >> (z == w1), name="indicator_constr1")
m.addConstr((b == 0) >> (z == w2), name="indicator_constr2")
